# Gibbsサンプリング

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

rng = np.random.default_rng()

## 目的分布: $p(\boldsymbol{z})$

Toy sample(バカみてーな例)として, 以下のような確率分布$p(z_1, z_2, z_3)$を考える:

$$
p(z_1, z_2, z_3) = 48 z_1 z_2 z_3
$$

ただし, $0 \leq z_1 \leq z_2 \leq z_3 \leq 1$であり, $z_1, z_2, z_3$は独立ではない. 周辺確率は,

$$
\begin{align}
p(z_1, z_2) &= \int_{z_2}^{1} p(z_1, z_2, z_3) \hspace{1mm} dz_3 = 24z_1z_2(1-z_2^2)\\
p(z_1, z_3) &= \int_{z_1}^{z_3} p(z_1, z_2, z_3) \hspace{1mm} dz_2 = 24z_1z_3(z_3^2-z_1^2)\\
p(z_2, z_3) &= \int_{0}^{z_2} p(z_1, z_2, z_3) \hspace{1mm} dz_1 = 24z_2^3z_3
\end{align}
$$

条件付き確率は,

$$
\begin{align}
p(z_1 | z_2, z_3) &= \frac{p(z_1, z_2, z_3)}{p(z_2, z_3)} = \frac{2z_1}{z_2^2} \\
p(z_2 | z_1, z_3) &= \frac{p(z_1, z_2, z_3)}{p(z_1, z_3)} = \frac{2z_2}{z_3^2-z_1^2} \\
p(z_3 | z_1, z_2) &= \frac{p(z_1, z_2, z_3)}{p(z_1, z_2)} = \frac{2z_3}{1-z_2^2} \\
\end{align}
$$

各条件付き確率の累積分布は,

$$
\begin{align}
P(z_1 | z_2, z_3) &= \int_0^{z_1} p(\zeta_1 | z_2, z_3) \hspace{1mm} d \zeta_1 = \frac{z_1^2}{z_2^2} \\
P(z_2 | z_1, z_3) &= \int_{z_1}^{z_2} p(\zeta_2 | z_1, z_3) \hspace{1mm} d \zeta_2 = \frac{z_2^2 - z_1^2}{z_3^2 - z_1^2} \\
P(z_3 | z_1, z_2) &= \int_{z_2}^{z_3} p(\zeta_3 | z_1, z_2) \hspace{1mm} d \zeta_3 = \frac{z_3^2 - z_2^2}{1 - z_2^2}
\end{align}
$$

累積分布の逆関数は,

$$
\begin{align}
P^{-1}(z_1 | z_2, z_3) &= z_2 \sqrt{x}\\
P^{-1}(z_2 | z_1, z_3) &= \sqrt{(z_3^2 - z_1^2)x + z_1^2}\\
P^{-1}(z_3 | z_1, z_2) &= \sqrt{(1 - z_2^2)x + z_2^2}
\end{align}
$$

なので, $x \sim \text{Uni}(0,1)$をサンプリングしてきて上のように変数変換すれば良い.

In [ ]:
def sample1(x, z_2, z_3):
    return z_2 * np.sqrt(x)

def sample2(x, z_1, z_3):
    return np.sqrt((z_3**2 - z_1**2)*x + z_1**2)

def sample3(x, z_1, z_2):
    return np.sqrt((1 - z_2**2)*x + z_2**2)

## ギブスサンプリングの手順

1. $\boldsymbol{z} = (z_1, z_2, z_3)$に初期値$\boldsymbol{z}^{(0)} = (z_1^{(0)}, z_2^{(0)}, z_3^{(0)})$を与える.
2. 各$\tau = 1, 2, \cdots, T$について, 以下のSTEP 3, 4, 5を行う.
3. 各$i=1, 2, 3$について, 以下のSTEP 4, 5を行う.
4. $z_i^{(\tau)}$を$p(z_i | \boldsymbol{z}_{\setminus i}^{(\tau-1)})$からサンプリングする. ただし$\boldsymbol{z}_{\setminus i}$は$z_i$以外の$\boldsymbol{z}$の要素.
5. 得られた$z_i^{(\tau)}$と$\boldsymbol{z}_{\setminus i}^{(\tau-1)}$を組み合わせたもの$\boldsymbol{z}^*$をサンプルとして保存する.

実際は初期値の影響が小さくなるまでSTEP3, 4, 5を行なってから(Burn-in process)サンプルを収集する.

In [ ]:
z = np.array([0.5, 0.5, 0.5])

ITER = 1000 # iteration to burn-in

for _ in range(ITER):
    z[0] = sample1(rng.random(), z[1], z[2])
    z[1] = sample2(rng.random(), z[0], z[2])
    z[2] = sample3(rng.random(), z[0], z[1])

size = 3 * 100000
sample = np.zeros((size, 3))

for i in range(100000):
    z[0] = sample1(rng.random(), z[1], z[2])
    sample[3*i] = z
    z[1] = sample2(rng.random(), z[0], z[2])
    sample[3*i + 1] = z
    z[2] = sample3(rng.random(), z[0], z[1])
    sample[3*i + 2] = z

## 答え合わせ

各変数における周辺確率は,

$$
\begin{align}
p(z_1) &= 6z_1^5 - 12z_1^3 + 6z_1 \\
p(z_2) &= 12z_2^3 - 12z_2^5 \\
p(z_3) &= 6 z_3^5
\end{align}
$$

よって, 各変数の期待値は,

$$
\begin{align}
\mathbb{E}(z_1) &= \frac{16}{35} \\
\mathbb{E}(z_2) &= \frac{24}{35} \\
\mathbb{E}(z_3) &= \frac{6}{7}
\end{align}
$$

である. 比較してみよう.

In [ ]:
mu = np.mean(sample, axis=0)
print("平均1, 平均2, 平均3")
print(16/35, 24/35, 6/7)
print(mu[0], mu[1], mu[2])

平均1, 平均2, 平均3
0.45714285714285713 0.6857142857142857 0.8571428571428571
0.4554351445437539 0.6841677908959115 0.8564906944855561


良い感じ